# ОПИСАНИЕ
В первой части мы провели анализ данных и построение коллаборативной рекомендательной системы. В этой части мы построим** content base** рекомендательную систему, основанную на семантическом сходстве описания видео и тегов. Обалсть применения - рекомендация похожих видео из канала того автора, видео которого только что посмотрел пользователь.




# ИМПОРТ



In [1]:
# A dependency of the preprocessing for BERT inputs
!pip install -q -U tensorflow-text

In [2]:
import os
import re
import time
import pandas as pd
import numpy as np
import pprint
import tempfile
from typing import Dict, Text

from sklearn.neighbors import NearestNeighbors
import tensorflow as tf

from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.layers import Embedding, Flatten, Input, concatenate, Reshape
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam, Adadelta
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
import tensorflow_hub as hub
import tensorflow_text as text

import seaborn as sns
from matplotlib import pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
PATH = '/content/drive/MyDrive/Colab Notebooks/HACKATHONS/Видеопереворот/Рекомендательная система/files/'

# Обезличенная база данных взаимодействия пользователей с видео-контентом (лайки, просмотры, комментарии, загрузки и пр.) 
video_dataset = 'video_dataset.csv'

# Обезличеная база данных видеоконтента 
concurs_data = 'concurs_data.csv'

# ДАННЫЕ

In [5]:
df_concurs = pd.read_csv(PATH + concurs_data)

In [6]:
df_concurs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   _key           100000 non-null  int64 
 1   channel_name   99963 non-null   object
 2   comment_count  100000 non-null  int64 
 3   description    95205 non-null   object
 4   dislike_count  100000 non-null  int64 
 5   duration       100000 non-null  int64 
 6   fulltitle      100000 non-null  object
 7   like_count     100000 non-null  int64 
 8   tags           100000 non-null  object
 9   upload_date    100000 non-null  int64 
 10  uploader_url   99998 non-null   object
 11  view_count     100000 non-null  int64 
dtypes: int64(7), object(5)
memory usage: 9.2+ MB


In [7]:
df_concurs.head()

,_key,channel_name,comment_count,description,dislike_count,duration,fulltitle,like_count,tags,upload_date,uploader_url,view_count
0,21350050,ZO'R TV,135,Официальный сайт: http://www.zortv.uz\nZO'RTV ...,0,1742,Bitta savol bor 1- son (10.07.2018),437,"[""zo'r tv"" 'zor tv' 'зур тв' 'зор тв']",1531170000,http://www.youtube.com/channel/UCisKdSbTsPt16y...,35735
1,71990270,Мир Белогорья,0,ЧП произошло в Краснояружском Староселье. Напо...,0,36,Супружеская пара подорвалась на боеприпасе неу...,23,['белгород' 'белгородская область' 'белгород н...,1669064400,http://www.youtube.com/@mirbelogor,931
2,61570019,Спорт-Экспресс,45,"Российский лыжник рассказал, каково чувствоать...",0,387,"Сергей Устюгов: «На горе может выключить так, ...",644,[],1578085200,http://www.youtube.com/user/sportexpress,30898
3,22003727,Extra News,0,Министр иностранных дел Венгрии Петер Сийярто ...,0,155,Венгрия удостоила Мантурова высшей госнаграды ...,1,['Новости' 'События' 'политика' 'народы'],1625691600,http://www.youtube.com/channel/UCYasXPsHhTVsn5...,63
4,28432077,Твой Дом - Тюрьма,0,Обязательно подпишитесь на наш канал! Воры в з...,0,220,"Коронован ""Дедом Хасаном"", воевал с Монголом. ...",63,['воры в законе 2019' 'криминал' 'киллеры' 'бл...,1560805200,http://www.youtube.com/user/max17111,18455


### Количество уникальных Каналов

In [8]:
df_concurs['channel_name'].unique().shape

(7463,)

## Отберём популярные видео для 1000 самых популярных каналов

### 1000 Самых популярных каналов

In [9]:
df_channel_group = df_concurs.groupby(['channel_name']).agg({'view_count':'sum'})
df_channel_group.columns = ['sum']
df_channel_popular = df_channel_group.sort_values(by='sum', ascending=False)[0:10]
df_channel_popular

,sum
channel_name,
Get Movies,4866339822
Маша и Медведь,1364028416
wilimovich,670513835
Три Кота,212514768
WOW TV,176134149
Watch Me,173489781
Nick Jr. Россия,172442449
HammAli &amp; Navai,168131709
ТВ Деткам,161131947


### Отберём видео для этих каналов

In [10]:
df_channel_popular.index

Index(['Get Movies', 'Маша и Медведь ', 'wilimovich', 'Три Кота', 'WOW TV',
       'Watch Me', 'Nick Jr. Россия', 'HammAli &amp; Navai', 'ТВ Деткам',
       'СТС'],
      dtype='object', name='channel_name')

In [11]:
df_popular_index = df_concurs['channel_name'].isin(df_channel_popular.index)
df_popular = df_concurs[df_popular_index]
df_popular.shape

(351, 12)

In [12]:
df_popular.head(20)

,_key,channel_name,comment_count,description,dislike_count,duration,fulltitle,like_count,tags,upload_date,uploader_url,view_count
640,29233156,СТС,145,"Шоу, в котором герои учатся справляться с жизн...",0,775,#вмаскешоу | Выпуск 8,1786,['стс' 'ctc' 'стс медиа' 'ctc media' 'ctctv' '...,1590699600,http://www.youtube.com/user/ctctv,128398
1714,72497233,СТС,742,Подпишись на новые серии: http://bit.ly/CTCTVS...,0,1263,Семейный бизнес - Сезон 1 Серия 1 - русская ко...,20055,['Семейный бизнес' 'семейный бизнес сериал' 'с...,1450213200,http://www.youtube.com/@ctctv,2900661
2459,29366778,wilimovich,61,"Nevertheless, it seems to me that Unimog on th...",0,307,RC Cars MUD OFF Road – Unimog 6x6 VS Unimog 4x...,650,['rc truck' 'rc cars' 'rc muddy' 'mud racing t...,1584133200,http://www.youtube.com/user/wilimovich,235275
2913,29204822,СТС,13,Подпишись на канал СТС: https://www.youtube.co...,0,473,"Импровизация DJ Smash | Слава Богу, ты пришел!",187,['слава богу ты пришёл' 'слава богу ты пришел ...,1559422800,http://www.youtube.com/user/ctctv,20783
3116,28973418,ТВ Деткам,0,Чуча Веселкин отправился на прогулку в джунгли...,0,184,Развивающий мультфильм. Чуча Веселкин в джунгл...,139,['мультфильмы' 'мультфильмы для детей' 'развив...,1449090000,http://www.youtube.com/user/tvdetkam,130283
3702,21931631,СТС,7,Шесть актеров мастерски разыгрывают миниатюрны...,0,1416,6 Кадров | Сезон 8 | Серия 283,291,['6 кадров' 'шесть кадров' 'стс' 'хорошие шутк...,1486501200,http://www.youtube.com/user/ctctv,66243
4172,28752084,Watch Me,0,Смотри как я / Watch Me – Сезон 4 Серия 22. Но...,0,1068,ТЕЛЕПАТИЯ ЛУЧШЕЙ ПОДРУЖКИ челлендж! СЛЕНДЕРМЕН...,24050,['телепатия челлендж' 'слендермен' 'ксюша мака...,1583182800,http://www.youtube.com/channel/UCGTl_sQxago30b...,1368710
4299,21482515,Nick Jr. Россия,0,У щенков новое задание! Нужно раскрасить пасха...,0,255,Щенячий патруль | Пасха уже тут | Nick Jr. Россия,71741,['Щенячий патруль' 'щенячий патруль новые сери...,1539723600,http://www.youtube.com/channel/UCSLZXHSv1aQDNe...,26958793
4442,68141145,СТС,0,Заходи в магазин ТРИ КОТА: \nhttps://market.ya...,0,333,Три кота | Серия 14 | Говорящая птица,11355,['три кота' 'три кота мультики' 'три кота новы...,1467234000,http://www.youtube.com/@ctctv,2911814
4952,29257254,WOW TV,0,Слушай и загружай сингл: https://wmr.lnk.to/po...,0,188,GAYAZOV$ BROTHER$ - По синей грусти | Official...,63681,['wow tv' 'wowtvnow' 'вов тв' 'клип' 'видео' '...,1581541200,http://www.youtube.com/user/wowtvnow,13322850


## Отберём 1000 популярных видео для холодного старта

In [13]:
df_concurs_1000 = df_concurs.sort_values(by='view_count', ascending=False)[0:1000]

In [14]:
df_concurs_1000.head()

,_key,channel_name,comment_count,description,dislike_count,duration,fulltitle,like_count,tags,upload_date,uploader_url,view_count
79153,21688483,Get Movies,0,День Варенья https://youtu.be/jHHP8uXO4y8\nВсе...,0,412,Маша и Медведь (Masha and The Bear) - Маша плю...,7849122,['Мультфильм' 'новый сезон' 'новые серии' 'нов...,1327953600,http://www.youtube.com/user/getmovies,4515610219
69863,29175932,Маша и Медведь,0,НОВЫЙ СЕЗОН! 🔥 🐻 Крути педали 🚵‍♂️ https://you...,0,408,"Маша и Медведь - Спи, моя радость, усни! (Сери...",1102692,['МашаМедведь' 'Мультфильм' 'новый сезон' 'mas...,1486069200,http://www.youtube.com/user/MashaMedvedTV,431690463
12905,21202459,wilimovich,0,More interesting videos http://www.youtube.com...,0,304,RC Cars MUD OFF Road — Land Rover Defender 90 ...,727684,['rc mud' 'rc muddy' 'racing' 'action' 'crash'...,1495832400,http://www.youtube.com/user/wilimovich,358717167
10902,21522024,wilimovich,0,More interesting videos http://www.youtube.com...,0,314,RC Cars 4x4 Sands Storm Racing and MUD Action ...,821021,['wilimovich' 'WLtoys' 'Axial' 'Wraith' 'Vater...,1497042000,http://www.youtube.com/user/wilimovich,307084502
69832,61484691,Маша и Медведь,0,ПРЕМЬЕРА! 🔥🍒 Калинка-Малинка 🍓 https://youtu.b...,0,411,Маша и Медведь - Следы невиданных зверей🐾 (Сер...,671923,['МашаМедведь' 'детям' 'Мультфильм' 'новый сез...,1369598400,http://www.youtube.com/user/MashaMedvedTV,213488204


# "CONTENT BASED" РЕКОМЕНДАТЕЛЬНАЯ СИСТЕМА НА ОСНОВЕ СЕМАНТИЧЕСКОГО СХОДСТВА
Предложение сделать рекомендательную систему на основе семантического сходства текстов "**description**", "**fulltitle**" и "**tags**". Для этого преобразуем тест в вектора BERT и кластеризуем методом ближайших соседей. В качестве рекомендаций - будем выдавать ближайших соседей. Эти вектора так же пригодятся в дальнейшем - для поиска по текстовому описанию.

## Загрузка модели BERT
Мы загрузим две модели, одну для предварительной обработки, а другую для кодирования. Ссылки на модели приведены ниже.

In [15]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4", trainable=True)

## Инициализация слоев BERT
В приведенном ниже коде мы создаем входной слой, используя tf.keras.layers.Input метод. Мы будем использовать в **preprocessed_text** качестве входных данных для этого слоя.

Затем **bert_encoder** функция преобразует предварительно обработанный текст в embadding векторы. Это будет выход этого слоя. Затем **outputs** они будут переданы в слои нейронной сети.

In [16]:
text_input = Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

### Проверим работу

In [17]:
p_text = bert_preprocess(['text'])
v = bert_encoder(p_text)

In [18]:
v['pooled_output'][0]

<tf.Tensor: shape=(768,), dtype=float32, numpy=
array([ 2.70360082e-01, -1.51426017e-01,  2.02479690e-01,  1.36259735e-01,
       -1.11354217e-01,  2.72155672e-01,  1.81260586e-01,  2.49676630e-01,
       -4.55122530e-01,  2.98245847e-01,  2.79911589e-02, -6.94658905e-02,
       -1.89139619e-01, -1.81658924e-01,  3.23850587e-02, -1.17484927e-01,
        4.35243487e-01,  8.09016544e-03, -1.77940235e-01, -4.19278741e-01,
       -9.99817014e-01, -2.44627044e-01, -3.81946921e-01, -2.06391916e-01,
       -3.17560732e-01,  1.76368654e-01, -6.06947541e-02,  2.72821367e-01,
        3.63530397e-01, -1.30347505e-01,  3.64892185e-04, -9.99835253e-01,
        4.42686290e-01,  5.91138065e-01,  2.33706027e-01, -8.11598971e-02,
        1.85111642e-01,  1.85093209e-01,  3.22861195e-01, -3.63657027e-01,
        2.37809997e-02, -1.49478361e-01,  1.35586590e-01,  1.46379545e-01,
        1.47330984e-01, -3.42431307e-01, -8.86009410e-02,  6.14471436e-02,
       -3.10208708e-01,  5.89239858e-02, -1.73562448

## Функции

### Функция создадия BERT векторов
В рабочей версии - вектор должен создаваться при загрузке видео, а не на этом шаге - для того, что бы сократить количество операций.

In [19]:
def get_vector(df_current):
  time_start = time.time()

  key_list = []
  vector_list = []
  for i in range(df_current.shape[0]):
    df_txt = df_current.iloc[i]

    key = df_txt['_key']

    if pd.notna(df_txt['description']):
      description = re.sub(r'[^\w\d\s\-]', '', df_txt['description'])
      description_c = re.sub(r'\s{2,}', ' ', description)
      if len(description_c) > 2:
        description_v = bert_encoder(bert_preprocess([description_c]))['pooled_output'][0]
        key_list.append(key)
        vector_list.append(description_v)

    if pd.notna(df_txt['fulltitle']):
      fulltitle = re.sub(r'[^\w\d\s\-]', '', df_txt['fulltitle'])
      fulltitle_c = re.sub(r'\s{2,}', ' ', fulltitle)
      if len(fulltitle_c) > 2:
        fulltitle_v = bert_encoder(bert_preprocess([fulltitle_c]))['pooled_output'][0]
        key_list.append(key)
        vector_list.append(fulltitle_v)

    if pd.notna(df_txt['tags']):
      tags = re.sub(r'[^\w\d\s\-]', '', df_txt['tags'])
      tags_c = re.sub(r'\s{2,}', ' ', tags)
      if len(tags_c) > 2:
        tags_v = bert_encoder(bert_preprocess([tags_c]))['pooled_output'][0]
        key_list.append(key)
        vector_list.append(tags_v)

  df_key = pd.DataFrame(key_list, columns=['key'])

  time_delta = round(time.time() - time_start)
  # print(f'Время выполнения: {time_delta}c')

  return df_key, vector_list

### Функция получения рекомендаций

In [20]:
def get_recom(df_popular, key):
  # Получаем Series нашего видео

  df_test = df_popular[['_key', 'channel_name', 'fulltitle', 'description', 'tags']][df_popular['_key'] == key]
  channel_name = df_test['channel_name'].iloc[0]
  
  # Получим вектор поля 'fulltitle' для только что просмотренного видео.
  test = re.sub(r'[^\w\d\s\-]', '', df_test['fulltitle'].iloc[0])
  test = re.sub(r'\s{2,}', ' ', test)
  test_v = bert_encoder(bert_preprocess([test]))['pooled_output'][0]

  # Получаем видео кандидатов
  df_candidates = df_popular[df_popular['channel_name'] == channel_name]

  # Получаем датафрейм 
  # ВАЖНО! В рабочей версии вектора должны быть заранее сформированы и внесены в базу данных
  # Но так как сформированных векторов нет, мы их формируем в процессе выполнения
  df_key, vector_list = get_vector(df_candidates)

  # Ближайшие соседи
  neigh = NearestNeighbors(n_neighbors=11, n_jobs=-1, metric='euclidean') 
  neigh.fit(vector_list)

  # Находим ближайших соседей и их индексы в списке
  res = neigh.kneighbors([test_v], return_distance=True)
  idx = res[1][0]
  candidates_key = df_key.iloc[idx]

  # Получаем кандидатов и не учитываем только что просмотренное видео
  df_recom = df_candidates[df_candidates['_key'].isin(candidates_key['key']) & (df_candidates['_key'] != key)]

  # print(df_key.head())
  # print(len(vector_list))
  # print(vector_list[0])

  return df_recom[['_key', 'channel_name', 'fulltitle', 'description', 'tags']]


## Протестируем

### Тест 1

In [21]:
key = 68141145
df_recom = get_recom(df_popular, key)

df_test = df_popular[df_popular['_key'] == key]
test_title = df_test['fulltitle'].iloc[0]
print('ПРОСМОТРЕННОЕ ВИДЕО:')
print(test_title)
print()

print('РЕКОМЕНДОВАННЫЕ ВИДЕО:')
for i in range(df_recom.shape[0]):
  print(df_recom['fulltitle'].iloc[i])

ПРОСМОТРЕННОЕ ВИДЕО:
Три кота | Серия 14 | Говорящая птица

РЕКОМЕНДОВАННЫЕ ВИДЕО:
Три кота | Сборник потрясающих серий | Мультфильмы для детей😃
Три кота | Серия 9 | Детектив
Кино в 23:45 | Моя супербывшая
Три Кота | Сборник невероятных открытий | Мультфильмы для детей 2020
Папины дочки | Сезон 8 | Серия 164
Три Кота | Сборник артистичных серий | Мультфильмы для детей 👨🌂🎭
Восьмидесятые | Серия 72
Молодежка | Сезон 1 | Серия 13
Три кота | Сборник | Серия 71 - 75
Три кота | Серия 23 | Снежные горки


### Тест 2

In [22]:
key = 29204822
df_recom = get_recom(df_popular, key)

df_test = df_popular[df_popular['_key'] == key]
test_title = df_test['fulltitle'].iloc[0]
print('ПРОСМОТРЕННОЕ ВИДЕО:')
print(test_title)
print()

print('РЕКОМЕНДОВАННЫЕ ВИДЕО:')
for i in range(df_recom.shape[0]):
  print(df_recom['fulltitle'].iloc[i])

ПРОСМОТРЕННОЕ ВИДЕО:
Импровизация DJ Smash | Слава Богу, ты пришел!

РЕКОМЕНДОВАННЫЕ ВИДЕО:
Кино на майские — лишь для тебя!
МастерШеф. Дети: новое испытание на 15 минут
Принц Сибири: девушки в предвкушении!
Все против Макеева | Молодежка Лёд и пламя
Молодёжка: Антипова посадят в тюрьму?
Импровизация Макса +100500 | Слава Богу, ты пришел!
Импровизация Ингеборги Дапкунайте | Слава Богу, ты пришел!
Импровизация Андрея Рожкова | Слава Богу, ты пришёл!
Побег идёт не по плану | Мамы чемпионов
AMORE - Детский хор Светлакова | Слава Богу, ты пришел!


### Тест 3

In [23]:
key = 21482515
df_recom = get_recom(df_popular, key)

df_test = df_popular[df_popular['_key'] == key]
test_title = df_test['fulltitle'].iloc[0]
print('ПРОСМОТРЕННОЕ ВИДЕО:')
print(test_title)
print()

print('РЕКОМЕНДОВАННЫЕ ВИДЕО:')
for i in range(df_recom.shape[0]):
  print(df_recom['fulltitle'].iloc[i])

ПРОСМОТРЕННОЕ ВИДЕО:
Щенячий патруль | Пасха уже тут | Nick Jr. Россия

РЕКОМЕНДОВАННЫЕ ВИДЕО:
Щенячий патруль | В поиске рюкзака | Nick Jr. Россия
День рождения Бульки | Nick Jr. Россия
Щенячий патруль | Лучшие зимние приключения! ❄️ | Nick Jr. Россия
Расти-механик | Ночные огни Расти | Nick Jr. Россия
44 котёнка | Кис-кис-мобиль | Nick Jr. Россия
Щенячий патруль | Щенки и котята-шалуны 🐱 | Nick Jr. Россия
Даша и друзья | Скрипка Эммы | Nick Jr. Россия
Даша и друзья | Поход с друзьями 🔥| Nick Jr. Россия
Щенячий патруль | Лучший пожарник | Nick Jr. Россия
Щенячий патруль | Самый лучший день! - часть 3 | Nick Jr. Россия


# ВЫВОД
Построение рекомендательной системы основаной на семантическом сходстве контента на основе сравнения близости векторов BERT даёт очень хорошие результаты, которые по качеству опережают подобные решения, основанные на TF IDF